최단경로: 가장 짧은 경로를 찾는 알고리즘  
* 다익스트라 최단경로 알고리즘 
* 플로이드 워셜 알고리즘    

1. 다익스트라 최단경로 알고리즘
   - 음의 간선 없을 때 정상작동.  
   - 음의 간선: 0보다 작은 값을 가지는 간선을 의미
   - 현실의 간선은(길) 음의 간선으로 표현되지 않아 다익스트라는 실제로 gps 소프트웨어 기본 알고리즘으로 채택된다.  
   - 가장 비용이 적은 노드를 선택해서 임의의 과정을 반복(그리디)  
   1) 출발노드 설정  
   2) 최단거리 테이블 초기화  
   3) 방문하지 않은 노드 중에서 최단 거리가 가장 짧은 노드를 선택한다.  
   4) 해당 노드를 거쳐 다른 노드로 가는 비용을 계산하여 최단 거리 테이블을 갱신한다.  
   5) 3,4를 반복  
  
   - 각 노드에 대한 현재까지의 최단거리 정보를 1차원 리스트에 저장하며 리스트 계속 갱신.
   - 현재 처리하고 있는 노드를 기준으로 주변 간선을 확인.
   - 방문하지 않은 노드 중에서 현재 최단거리가 가장 짧은 노드를 확인해 그 노드에 대해 4번 수행.(그리디)

In [6]:
# 예제 9-1. 간단한 다익스트라 알고리즘 소스코드 ?? input error

import sys

input = sys.stdin.readline
INF = int(1e9) # infinite로 10억 설정

# 노드의 개수, 간선의 개수 입력받기
n, m = map(int, input().split())
# 시작노드 번호 입력받기
start = int(input())
# 각 노드에 연결되어 있는 노드에 대한 정보 담는 리스트 만들기
graph = [[] for i in range(n+1)]
# 방문한 적이 있는지 체크하는 목적의 리스트를 만들기
visited = [False] * (n+1)
# 최단 거리 테이블을 모두 무한으로 초기화
distance = [INF] * (n+1)

# 모든 간선 정보를 입력받기
for _ in range(m):
    a, b, c = map(int, input().split())
    # a번 노드에서 b번 노드로 가는 비용이 c
    graph[a].append((b,c))
    
# 방문하지 않은 노드 중에서, 가장 최단 거리가 짧은 노드의 번호를 반환
def get_smallest_node():
    min_value = INF
    index = 0 # 가장 최단거리가 짧은 노드(인덱스)
    for i in range(1, n+1):
        if distance[i] < min_value and not visited[i]:
            min_value = distance[i]
            index = i
    return index

def dijkstra(start):
    # 시작 노드에 대해서 초기화
    distance[start] = 0
    visited[start] = True
    for j in graph[start]:
        distance[j[0]] = j[1]
    # 시작 노드를 제외한 전체 n - 1개의 노드에 대해 반복
    for i in range(n-1):
        # 현재 최단 거리가 가장 짧은 노드를 꺼내서, 방문 처리
        now = get_smallest_node()
        visited[now] = True
        # 현재 노드와 연결된 다른 노드를 확인
        for j in graph[now]:
            cost = distance[now] + j[1]
            # 현재 노드를 거쳐서 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[j[0]]:
                distance[j[0]] = cost

# 다익스트라 알고리즘 수행
dijkstra(start)

# 모든 노드로 가기 위한 최단 거리를 출력
for i in range(1, n+1):
    # 도달할 수 없는 경우, 무한(INFINITY)이라고 출력
    if distance[i] == INF:
        print("INFINITY")
    # 도달할 수 있는 경우 거리를 출력
    else:
        print(distance[i])

ValueError: not enough values to unpack (expected 2, got 0)

2. 개선된 다익스트라 알고리즘  
    * 다익스트라보다 시간 더 짧음  
    * 힙 자료구조 사용. 특정 노드까지의 최단 거리에 대한 정보를 힙에 담아서 처리.  
    * 힙 자료구조: 큐를 구현하기 위한 자료구조.  
    * 큐: 가장 먼저 삽입된 데이터를 가장 먼저 삭제한다.(<-> 스택)  
    * 우선순위 큐: 우선순위가 가장 높은 데이터를 가장 먼저 삭제한다.
        - PriorityQue, heapq
        - 최단거리가 가장 짧은 노드를 선택하는 과정을 다익스트라 최단경로 함수 안에서 우선순위 큐를 이용하는 방식으로 대체
        - 현재 가장 가까운 노드를 저장하기 위한 목적으로 추가적 이용.


In [8]:
# 예제 9-2.py 개선된 다익스트라 알고리즘 소스코드

import heapq
import sys
input = sys.stdin.readline
INF = int(1e9)

# 노드의 개수, 간선의 개수 입력받기
n, m = map(int, input().split())
# 시작 노드 번호를 입력받기
start = int(input())
# 각 노드에 연결되어 있는 노드에 대한 정보를 담는 리스트 만들기
graph = [[] for i in range(n+1)]
# 최단 거리 테이블을 모두 무한으로 초기화
distance = [INF] * (n+1)

# 모든 간선 정보를 입력받기
for _ in range(m):
    a, b, c = map(int, input().split())
    # a번 노드에서 b번 노드로 가능 비용이 c라는 의미
    graph[a].append((b,c))
    
def dijkstra(start):
    q = []
    # 시작 노드로 가기 위한 최단 경로는 0으로 설정하여, 큐에 삽입
    heapq.heappush(q, (0, start))
    distance[start] = 0
    while q: # 큐가 비어있지 않다면
        # 가장 최단 거리가 짧은 노드에 대한 정보 꺼내기
        dist, now = heapq.heappop(q)
        # 현재 노드가 이미 처리된 적이 있는 노드라면 무시
        if distance[now] < dist:
            continue
        # 현재 노드와 연결된 다른 인접한 노드들을 확인
        for i in graph[now]:
            cost = dist + i[1]
            # 현재 노드를 거쳐서 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[i[0]]:
                distance[i[0]] = cost
                heapq.heappush(q, (cost, i[0]))
                
dijkstra(start)

# 모든 노드로 가기 위한 최단거리를 출력
for i in range(1, n+1):
    # 도달할 수 없는 경우, 무한이라고 출력
    if distance[i] == INF:
        print("INFINITY")
    # 도달할 수 있는 경우 거리를 출력
    else:
        print(distance[i])
        
        

ValueError: not enough values to unpack (expected 2, got 0)

3. 플로이드 워셜 알고리즘
    * 모든지점에서 다른 모든지점까지의 최단경로를 모두 구해야 하는 경우
    * 단계마다 거쳐가는 노드를 기준으로 알고리즘 수행. 하지만 매번 방문하지 않은 노드중에서 최단거리를 갖는 노드를 찾을 필요가 없다(다익스트라와 차이점)


In [1]:
# 9-3.py 플로이드 워셜 알고리즘 소스코드

INF = int(1e9) # 무한

# 노드의 개수 및 간선의 개수 입력
n = int(input())
m = int(input())
# 2차원 리스트(그래프 표현)를 만들고 모든 값을 무한으로 초기화
graph = [[INF] * (n + 1) for _ in range(n + 1)]

# 자기 자신에서 자기 자신으로 가는 비용은 0으로 초기화
for a in range(1, n+1):
    for b in range(1, n+1):
        if a == b:
            graph[a][b] = 0
            
# 각 간선에 대한 정보를 입력받아, 그 값으로 초기화
for _ in range(m):
    # A에서 B로 가는 비용은 C라고 설정
    a, b, c = map(int, input().split())
    graph[a][b] = c
    
# 점화식에 따라 플로이드 워셜 알고리즘을 수행
for k in range(1, n+1):
    for a in range(1, n+1):
        for b in range(1, n+1):
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])
            
# 수행된 결과를 출력
for a in range(1, n+1):
    for b in range(1, n+1):
        # 도달할 수 없는 경우, 무한이라고 출력
        if graph[a][b] == INF:
            print("INFINITY", end = " ")
        # 도달할 수 있는 경우 거리를 출력
        else:
            print(graph[a][b], end = " ")
print()

4
7
1 2 4
1 4 6
2 1 3
2 3 7
3 1 5
3 4 4
4 3 2
0 4 8 6 3 0 7 9 5 9 0 4 7 11 2 0 


실전예제1. 미래도시

In [3]:
# 9-4.py 답안예시

INF = int(1e9)

# 노드의 개수 및 간선의 개수 입력
n,m = map(int, input().split())
# 2차원 리스트 만들고 모든값 무한으로 초기화
graph = [[INF] * (n+1) for _ in range(n+1)]

# 자기자신에서 자기자신으로 가는 비용은 0으로 초기화
for a in range(1, n+1):
    for b in range(1, n+1):
        if a == b:
            graph[a][b] = 0
            
# 각 간선에 대한 정보 입력받아 그 값으로 초기화
for _ in range(m):
    # A와 B가 서로에게 가는 비용은 1이라고 설정
    a, b = map(int, input().split())
    graph[a][b] = 1
    graph[b][a] = 1
    
# 거쳐 갈 노드 X와 최종 목적지 노드 K를 입력받기
x,k = map(int, input().split())

# 점화식에 따라 플로이드 워셜 알고리즘을 수행
for k in range(1, n+1):
    for a in range(1, n+1):
        for b in range(1, n+1):
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])
            
# 수행된 결과를 출력
distance = graph[1][k] + graph[k][x]

# 도달할 수 없는 경우 -1 출력
if distance >= INF:
    print("-1")
# 도달할 수 있으면 최단거리 출력
else:
    print(distance)

5 7
1 2
1 3
1 4
2 4
3 4
3 5
4 5
4 5
3


실전예제2. 전보

In [18]:
import heapq
import sys

read = sys.stdin.readline
INF = int(1e9)

n, m, start = map(int, read().split())
graph = [[] for i in range(n+1)]
distance = [INF]* (n+1)

for _ in range(m):
    x, y, z = map(int, read().split())
    # x번 노드에서 y번 노드로 가는 비용이 z라는 의미
    graph[x].append((y,z))
    
def dijkstra(start):
    q =[]
    # 시작 노드로 가기 위한 최단 경로는 0으로 설정하여, 큐에 삽입
    heapq.heappush(q, (0,start))
    distance[start] = 0
    while q: # 큐가 비어있지 않다면
        # 가장 최단거리가 짧은 노드에 대한 정보를 꺼내기
        dist, now = heapq.heappop(q)
        if distance[now] < dist:
            continue
        # 현재 노드와 연결된 다른 인접한 노드들을 확인
        for i in graph[now]:
            cost = dist + i[1]
            # 현재 노드를 거쳐서, 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[i[0]]:
                distance[i[0]] = cost
                heapq.heappush(q, (cost,i[0]))
                
dijkstra(start)

# 도달할 수 있는 노드의 개수
count = 0
# 도달할 수 있는 노드 중 가장 멀리 있는 노드와의 최단거리
max_distance = 0
for d in distance:
    # 도달할 수 있는 노드인 경우
    if d != INF:
        count += 1
        max_distance = max(max_distance, d)
        
# 시작 노드는 제외해야 하므로 count - 1을 출력
print(count - 1, max_distance)

ValueError: not enough values to unpack (expected 3, got 0)

In [1]:
# 실전문제1. 플로이드
INF = int(1e9)

n = int(input())
m = int(input())
graph = [[INF] * (n+1) for _ in range(n+1)]

# 자기자신에서 자기자신으로 가는 비용은 0으로 초기화
for a in range(1, n+1):
    for b in range(1, n+1):
        if a == b:
            graph[a][b] = 0
            
# 각 간선에 대한 정보 입력받아 그 값으로 초기화
for _ in range(m):
    # A와 B가 서로에게 가는 비용은 1이라고 설정
    a, b, c = map(int, input().split())
    if c < graph[a][b]:
        graph[a][b] = c
    
# 점화식에 따라 플로이드 워셜 알고리즘을 수행
for k in range(1, n+1):
    for a in range(1, n+1):
        for b in range(1, n+1):
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])
            
for a in range(1, n+1):
    for b in range(1, n+1):
        if graph[a][b] == INF:
            print(0, end=" ")
        else:
            print(graph[a][b], end=" ")
    print()        

5
14
1 2 2
1 3 3
1 4 1
1 5 10
2 4 2
3 4 1
3 5 1
4 5 3
3 5 10
3 1 8
1 4 2
5 1 7
3 4 2
5 2 4
0 2 3 1 4 
12 0 15 2 5 
8 5 0 1 1 
10 7 13 0 3 
7 4 10 6 0 


In [3]:
# 실전문제2. 정확한 순위
INF = int(1e9)

n,m = map(int, input().split())

graph = [[INF] * (n+1) for _ in range(n+1)]

# 자기자신 비용은 0으로 설정
for a in range(1, n+1):
    for b in range(1, n+1):
        if a == b:
            graph[a][b] = 0
            
# 다른데로 가는 비용은 1로 설정
for _ in range(m):
    a, b = map(int, input().split())
    graph[a][b] = 1
    
for k in range(1, n+1):
    for a in range(1, n+1):
        for b in range(1, n+1):
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])

result = 0
for i in range(1, n+1):
    count = 0
    for j in range(1, n+1):
        if graph[i][j] != INF or graph[j][i] != INF:
            count += 1
        if count == n:
            result += 1
print(result)

6 6
1 5
3 4
4 2
4 6
5 2
5 4
1


In [4]:
# 실전문제 4. 숨박꼭질
import heapq
import sys
INF = int(1e9)

n,m = map(int, input().split())
start = 1

# 각 노드에 연결되어 있는 노드에 대한 정보를 담는 리스트 만들기
graph = [[] for i in range(n+1)]
# 최단 거리 테이블을 모두 무한으로 초기화
distance = [INF] * (n+1)

# 모든 간선 정보를 입력받기
for _ in range(m):
    a, b = map(int, input().split())
    # a번 노드에서 b번 노드로 가능 비용이 c라는 의미
    graph[a].append((b,1))
    graph[b].append((a,1))
    
def dijkstra(start):
    q = []
    # 시작 노드로 가기 위한 최단 경로는 0으로 설정하여, 큐에 삽입
    heapq.heappush(q, (0, start))
    distance[start] = 0
    while q: # 큐가 비어있지 않다면
        # 가장 최단 거리가 짧은 노드에 대한 정보 꺼내기
        dist, now = heapq.heappop(q)
        # 현재 노드가 이미 처리된 적이 있는 노드라면 무시
        if distance[now] < dist:
            continue
        # 현재 노드와 연결된 다른 인접한 노드들을 확인
        for i in graph[now]:
            cost = dist + i[1]
            # 현재 노드를 거쳐서 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[i[0]]:
                distance[i[0]] = cost
                heapq.heappush(q, (cost, i[0]))
                
dijkstra(start)

max_node = 0
max_distance = 0
result = []

for i in range(1, n+1):
    if max_distance < distance[i]:
        max_node = i
        max_distance = distance[i]
        result = [max_node]
    elif max_distance == distance[i]:
        result.append(i)
        
print(max_node, max_distance, len(result))        




6 7
3 6
4 3
3 2
1 3
1 2
2 4
5 2
4 2 3
